In [6]:
from typing import List
from models.game_state import GameState
from models.game_config import GameConfig, BaseLevel, PathConfig
from models.position import Position
from models.player_action import PlayerAction
from models.base import Base
from logic.strategy import get_base_distance
# import numpy as np
import math
import pandas as pd

In [13]:
base_levels = [
    BaseLevel(max_population=20,
        upgrade_cost=10,
        spawn_rate=1
    ),
    BaseLevel(
        max_population=40,
        upgrade_cost=20,
        spawn_rate=2
    ),
    BaseLevel(
        max_population=80,
        upgrade_cost=30,
        spawn_rate=3
    )
]

game_config = GameConfig(
    base_levels=base_levels,
    paths=PathConfig(
        grace_period=10,
        death_rate=1
    )
)

In [4]:
def get_enemy_values(otherBases: list[Base], config: GameConfig):
    data = {}
    enemyUid = []
    enemyGrowthRate = []
    enemyMaxPopulation = []
    enemyPopulation = []

    for targetBase in otherBases:
        enemyUid.append(targetBase.uid)
        enemyGrowthRate.append(config.base_levels[targetBase.level].spawn_rate)
        enemyMaxPopulation.append(config.base_levels[targetBase.level].max_population)
        enemyPopulation.append(targetBase.population)

    data["index"] = enemyUid
    data["growth_rate"] = enemyGrowthRate
    data["max_population"] = enemyMaxPopulation
    data["population"] = enemyPopulation

    return data

In [7]:
def get_enemy_distance(data: dict, otherBases: list[Base], ourBases: list[Base]) -> dict:
    for ourBase in ourBases:
        enemydist = []
        for targetBase in otherBases:
            
            #enemydist[targetBase.uid] = (get_base_distance(ourBase,targetBase) - config.paths.grace_period)
            enemydist.append(get_base_distance(ourBase,targetBase))
        data[ourBase.uid] = enemydist
    return data

In [9]:
def generate_base_costs(data: dict):
    allBases = pd.DataFrame(data)
    allBases = allBases.set_index(["index", "growth_rate", "max_population", "population"])
    return allBases

In [11]:
# add death_rate for euclidean distance to enemy
def add_death_rate(allBases_distanceCosts: pd.DataFrame, config: GameConfig):
    return ((allBases_distanceCosts.loc[:] - config.paths.grace_period).clip(lower=0) * config.paths.death_rate)

In [20]:
our_bases = [
    Base(uid=0, name='A', player=1, population=100, level=0, units_until_upgrade=9, position=Position(5, 0, 1)),
    Base(uid=1, name='B', player=1, population=100, level=0, units_until_upgrade=9, position=Position(20, 0, 1))
]

other_bases = [
    Base(uid=2, name='C', player=2, population=100, level=0, units_until_upgrade=9, position=Position(40, 0, 1)),
    Base(uid=3, name='D', player=3, population=100, level=0, units_until_upgrade=9, position=Position(80, 0, 1))
]

data = get_enemy_values(other_bases, game_config)
data = get_enemy_distance(data, other_bases, our_bases)
all_bases = generate_base_costs(data)

all_bases_distance_costs = all_bases.copy()

all_bases_distance_costs = add_death_rate(all_bases_distance_costs, game_config)
all_bases_distance_costs = add_gain_of_enemy(all_bases_distance_costs, all_bases)
all_bases_distance_costs = add_population_of_enemy_at_start(all_bases_distance_costs, all_bases)


all_bases_distance_costs

,,,,0,1
index,growth_rate,max_population,population,,
2,1,20,100,160,130
3,1,20,100,240,210
